In [1]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModel, AutoTokenizer
from transformers import T5Tokenizer, T5ForConditionalGeneration

import evaluate
import numpy as np
import pandas as pd
from datasets import Dataset, load_dataset
from transformers import DataCollatorForSeq2Seq
import evaluate

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import pandas as pd

In [4]:
df_t = pd.read_csv('spider_processed_Schemas_tablesSchemaSpider.csv')[['schema_id', 'table_name_original']]
df_q = pd.read_csv('spider_processed_spiderQueryData.csv')[['db_id', 'TQL', 'SQL']]

In [5]:
df_q

,db_id,TQL,SQL
0,department_management,How many heads of the departments are older th...,SELECT count(*) FROM head WHERE age > 56
1,department_management,"List the name, born state and age of the heads...","SELECT name , born_state , age FROM head ORD..."
2,department_management,"List the creation year, name and budget of eac...","SELECT creation , name , budget_in_billions ..."
3,department_management,What are the maximum and minimum budget of the...,"SELECT max(budget_in_billions) , min(budget_i..."
4,department_management,What is the average number of employees of the...,SELECT avg(num_employees) FROM department WHER...
...,...,...,...
10224,GreaterManchesterCrime,Where is the place with the largest number of ...,SELECT Location FROM GreaterManchesterCrime WH...
10225,GreaterManchesterCrime,If it is possible to increase the number of po...,SELECT Location FROM GreaterManchesterCrime GR...
10226,GreaterManchesterCrime,Which type of crime has the most prosecution c...,SELECT Type FROM GreaterManchesterCrime WHERE ...
10227,GreaterManchesterCrime,Which type of crime has the most investigation...,SELECT Type FROM GreaterManchesterCrime WHERE ...


In [6]:
df_t.groupby('schema_id')['table_name_original'].apply(list)

schema_id
GeoNuclearData                                       [nuclear_power_plants]
GreaterManchesterCrime                             [GreaterManchesterCrime]
Pesticide                                     [sampledata15, resultsdata15]
StudentMathScore          [FINREV_FED_17, NDECoreExcel_Math_Grade8, FINR...
TheHistoryofBaseball      [hall_of_fame, player_award, player_award_vote...
                                                ...                        
workshop_paper                           [workshop, submission, Acceptance]
world_1                   [city, sqlite_sequence, country, countrylanguage]
wrestler                                            [wrestler, Elimination]
wta_1                                          [players, matches, rankings]
yelp                      [business, category, user, checkin, neighbourh...
Name: table_name_original, Length: 174, dtype: object

In [7]:
df_tq = df_q.merge(
    df_t.groupby('schema_id')['table_name_original'].apply(list),
    how = 'left',
    left_on = 'db_id', right_on = 'schema_id'
)
df_tq.head()

,db_id,TQL,SQL,table_name_original
0,department_management,How many heads of the departments are older th...,SELECT count(*) FROM head WHERE age > 56,"[department, head, management]"
1,department_management,"List the name, born state and age of the heads...","SELECT name , born_state , age FROM head ORD...","[department, head, management]"
2,department_management,"List the creation year, name and budget of eac...","SELECT creation , name , budget_in_billions ...","[department, head, management]"
3,department_management,What are the maximum and minimum budget of the...,"SELECT max(budget_in_billions) , min(budget_i...","[department, head, management]"
4,department_management,What is the average number of employees of the...,SELECT avg(num_employees) FROM department WHER...,"[department, head, management]"


In [8]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers = 7)

INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [9]:
def get_tables(query, table_name_original):
    import re
    import nltk
    from nltk.corpus import stopwords
    sql_tokens = [token for token in re.sub('[^a-zA-Z_ ]', '', query).lower().split()
                              if ((token not in set(stopwords.words('english'))) and len(token) > 2)]
    actual_list = []
    for token in sql_tokens:
        for table_name in table_name_original:
            if token == table_name.lower():
                actual_list.append(token.lower())
                
    return ' '.join(list(set(actual_list)))

In [10]:
df_tq['table_names'] = df_tq.parallel_apply(
    lambda x: get_tables(x['SQL'], x['table_name_original']),
    axis = 1
)

In [11]:
df_sum1 = df_tq[['TQL', 'table_names']]

In [12]:
df_dataset = Dataset.from_pandas(df_sum1.head(10))

In [13]:
billsum = df_dataset.train_test_split(test_size=0.2)

In [14]:
billsum['test'][0]

{'TQL': 'How many heads of the departments are older than 56 ?',
 'table_names': 'head',
 '__index_level_0__': 0}

In [15]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

In [17]:
# def preprocess_function(examples):
#     inputs = examples["transcription"]
#     model_inputs = tokenizer(inputs, truncation=True, padding="longest")

#     labels = examples["summary"]
#     model_inputs["labels"] = tokenizer(labels, truncation=True, padding="longest").input_ids
#     return model_inputs

def preprocess_function(examples):
    inputs = [str(text) for text in examples["TQL"]]
    model_inputs = tokenizer(inputs, truncation=True, padding=False)

    labels = [str(summary) for summary in examples["table_names"]]
    model_inputs["labels"] = tokenizer(labels, truncation=True, padding=False).input_ids
    return model_inputs

In [18]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [20]:
rouge = evaluate.load("rouge")

In [22]:
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
#     result["gen_len"] = np.mean(prediction_lens)

#     return {k: round(v.mid.fmeasure * 100, 4) for k, v in result.items()}

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [24]:
training_args = Seq2SeqTrainingArguments(
    output_dir="PegasusMedicalSummary",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    push_to_hub=True,
    gradient_accumulation_steps=4,
)

In [25]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/home/jupyter/TQL/queryProcessing/PegasusMedicalSummary is already a clone of https://huggingface.co/jasmeeetsingh/PegasusMedicalSummary. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
# Start training
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
# Push the trained model to the Hugging Face Model Hub
trainer.push_to_hub()

In [ ]:
# Save the fine-tuned model
model.save_pretrained("PegasusMedicalSummary")

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, SummarizationPipeline

model_name = "jasmeeetSingh/PegasusMedicalSummary"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

summarizer = SummarizationPipeline(model=model, tokenizer=tokenizer)

In [ ]:
text = "1.  The left ventricular cavity size and wall thickness appear normal.  The wall motion and left ventricular systolic function appears hyperdynamic with estimated ejection fraction of 70% to 75%.  There is near-cavity obliteration seen.  There also appears to be increased left ventricular outflow tract gradient at the mid cavity level consistent with hyperdynamic left ventricular systolic function.  There is abnormal left ventricular relaxation pattern seen as well as elevated left atrial pressures seen by Doppler examination.,2.  The left atrium appears mildly dilated.,3.  The right atrium and right ventricle appear normal.,4.  The aortic root appears normal.,5.  The aortic valve appears calcified with mild aortic valve stenosis, calculated aortic valve area is 1.3 cm square with a maximum instantaneous gradient of 34 and a mean gradient of 19 mm.,6.  There is mitral annular calcification extending to leaflets and supportive structures with thickening of mitral valve leaflets with mild mitral regurgitation.,7.  The tricuspid valve appears normal with trace tricuspid regurgitation with moderate pulmonary artery hypertension.  Estimated pulmonary artery systolic pressure is 49 mmHg.  Estimated right atrial pressure of 10 mmHg.,8.  The pulmonary valve appears normal with trace pulmonary insufficiency.,9.  There is no pericardial effusion or intracardiac mass seen.,10.  There is a color Doppler suggestive of a patent foramen ovale with lipomatous hypertrophy of the interatrial septum.,11.  The study was somewhat technically limited and hence subtle abnormalities could be missed from the study.,"
summary = summarizer(text)
print(summary)